In [1]:
import os
from pathlib import Path
os.chdir("../")
print(Path.cwd())
fetchapi_url = "https://api.github.com/repos/{owner}/{repo}/contents/{path}".format(**{
  "owner": "mikepsinn",
  "repo": "FDAi",
  "path":""
})
fetchapi_url

/home/mymm.psu/FDAi-helper-agents


'https://api.github.com/repos/mikepsinn/FDAi/contents/'

In [3]:
import os, requests, re
from dotenv import load_dotenv  
print(load_dotenv(override=True))
print(os.environ['OPENAI_API_KEY'][-4:])

True
pzUy


In [11]:
from llama_index.core import VectorStoreIndex, SummaryIndex

from llama_index.readers.github import GithubRepositoryReader
from llama_index.readers.web import SimpleWebPageReader

from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=False
)

class web_agent:
  def __init__(self, url, index_regex:str=r'.+\.(md|txt)$'):
    self.initialized = False
    github_urls = self.fetch_github_repo_contents_urls(url)
    print(f"Extracted {len(github_urls)} in total from repo")
    github_urls = self.filter_urls(
      github_urls, regex=index_regex)
    print(f"Filtered to {len(github_urls)} urls")
    self._init_web_document_index(github_urls)
    
  def _init_web_document_index(self, urls:list, llm=True):
    documents = SimpleWebPageReader(html_to_text=True).load_data(
      urls
    )
    if llm:
      chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
      splitter = SentenceSplitter(chunk_size=1024)
      self.doc_index = DocumentSummaryIndex.from_documents(
          documents,
          llm=chatgpt,
          transformations=[splitter],
          response_synthesizer=response_synthesizer,
          # show_progress=True,
      )
    else:
      self.doc_index = SummaryIndex.from_documents(documents)
    self._init_query_engine(self.doc_index)
  
  def _init_query_engine(self, index):
    self.query_engine = index.as_query_engine(streaming=True)
    self.initialized = True
  
  def query(self, query):
    return self.query_engine.query(query)
  
  def filter_urls(
    self, urls, regex:str=None, limit:int=None):
    if regex: 
      pattern = re.compile(regex, re.IGNORECASE)
      urls=[
        l for l in urls if pattern.search(l)]
    if limit: urls=urls[:min(len(urls), limit)]
    return urls
    
  def fetch_github_repo_contents_urls(
    self, api_url,path=""):
    headers = {
        "Authorization": f"token {os.environ['GITHUB_TOKEN']}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(
        api_url+path, headers=headers
    )
    urls = []
        # Check if the request was successful
    if response.status_code == 200:
        contents = response.json()
        # Ensure contents is a list before iterating
        if not isinstance(contents, list):
            return urls

        for n, content in enumerate(contents):
            # print(f"Filename:{content.get('name','')}")
            # If it's a directory, recurse into it
            if content.get('type') == 'dir':
                urls.extend(self.fetch_github_repo_contents_urls(api_url+path, content.get('path')))
            else:
                # Append the file URL to the list
                urls.append(content.get('html_url', 'No URL found'))
        print(f"Extracted: {n} urls here")
    # else:
    #     # Print out the error message from GitHub API
    #     print(f"Error fetching contents: {response.status_code} - {response.json().get('message')}")
    return urls
  
  @property
  def _documents(self):
    if hasattr(self, 'documents'):
      return self.documents

class directory_agent(web_agent):
  def __init__(self, local_path, exts_filter:list=[".md", ".txt", ".yaml", ".json"]):
    self._init_web_document_index(local_path, exts_filter)
  
  def _init_web_document_index(
    self, path, exts_filter, llm=True, model="gpt-4"):
    self.documents = SimpleDirectoryReader(
      path, recursive=True, required_exts=exts_filter
    ).load_data()
    if llm:
      chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
      # splitter = SentenceSplitter(chunk_size=1024)
      self.doc_index = DocumentSummaryIndex.from_documents(
        self.documents,
        llm=chatgpt,
        # transformations=[splitter],
        response_synthesizer=response_synthesizer,
        # show_progress=True,
      )
    else:
      self.doc_index = SummaryIndex.from_documents(self.documents)
    self._init_query_engine(self.doc_index)

### Test

In [10]:
# query_agent = web_agent(fetchapi_url)
query_agent = directory_agent(
  "FDAi", exts_filter=[".md", ".txt", ".json"]
)

current doc id: cbb248ed-f52f-476b-b61a-be7536c23d35


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
print(len(query_agent.documents))
print(query_agent.documents[0])

In [4]:
prompts = [
  "What is the FDAi-act in terms of the key sections ?"
  "What is clinipedia?",
  "What is the FDAi repo about?"
]
prompt=prompts[0]

# flexible method
query_engine = query_agent.doc_index.as_query_engine(
  streaming=True, similarity_top_k=5)
streaming_response = query_engine.query(prompt)
# or
# streaming_response = query_agent.query(prompt)
streaming_response.print_response_stream()

NameError: name 'query_agent' is not defined

In [45]:
query_agent.doc_index

In [42]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever
)
if True:
    retriever = query_agent.doc_index.as_retriever(
        retriever_mode="llm",
    )
else:
    
    retriever = DocumentSummaryIndexLLMRetriever(
        query_agent.doc_index,
        choice_select_prompt=None,
        choice_batch_size=10,
        choice_top_k=1,
        format_node_batch_fn=None,
        parse_choice_select_answer_fn=None,
    )

In [46]:
retrieved_nodes = retriever.retrieve(prompt)
print(len(retrieved_nodes))

1


In [47]:
print(retrieved_nodes[0].node.get_text())

3. Clinipedia—The Wikipedia of Clinical Research

![clinipedia_globe_circle.png](docs/components/clinipedia/clinipedia.md)

The Clinipedia wiki should be a global knowledge repository containing the aggregate of all available data on the effects of every food, drug, supplement, and medical intervention on human health.

**👉 Learn More or Contribute to the Clinipedia**


## Experiment Approaches

In [ ]:
# Require the following
print(type(query_agent.document_store))


<class 'list'>


In [ ]:
query_agent.query_engine.query('What is FDAi project about?')

In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# Define tools

tools = [
    Tool(
       name = "LlamaIndex",
        func=lambda q: str(query_agent.query_engine.query(q)),
        description="context specific query engine",
        return_direct=True
    ),
]
#Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory( memory_key='chat_history', k=5, return_messages=True )
# Initialize agent with conversational memory
agent_executor = initialize_agent(
    tools, llm=ChatOpenAI(
        temperature=0.7, model_name='gpt-4'), agent="conversational-react-description", memory=conversational_memory)

# if I use my index
prompt = input("type prompt")
# What is the FDAi GitHub repoisitory about?
if prompt == 'thanks':
    print(f' ---------- Agent: chat is closed -------')
else:
    response = agent_executor.run(input=prompt)
    print(f'me : {prompt}')
    print(f'Agent : {response}')

/home/mmym_ezout_ezout_store_c_04fb7ek/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


me : What is the FDAi GitHub repoisitory about?
Agent : The FDA, or Food and Drug Administration, does not have an officially recognized GitHub repository as of my training data up to September 2021. If you're referring to a specific repository by a different name or acronym, could you please provide more details? In general, GitHub repositories are used to host and share code for various projects, so an "FDAi" repository might potentially be related to projects or initiatives associated with the FDA. However, without more specific details, it's hard to provide an accurate description.
